In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

# ResNet을 이용한 CNN
ResNet을 따라해보고 더 깊은 컨볼루션 신경망을 만들어 봅시다..!
---

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 30 #정확도를 올리고자 하면 에포크 수 늘리기!!
BATCH_SIZE = 128

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train = True,
                     download=True,
                     transform = transforms.Compose([
                         transforms.RandomCrop(32,padding=4),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5,0.5,0.5),
                                              (0.5,0.5,0.5))
                     ])),
    batch_size = BATCH_SIZE,shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=False,
                     transform = transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.5,0.5,0.5),
                                              (0.5,0.5,0.5))
                     ])),
    batch_size = BATCH_SIZE,shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./.data\cifar-10-python.tar.gz to ./.data


In [5]:
# ResNet은 여러개의 BasicBlock을 거쳐서 학습을 한다.
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride = stride,padding=1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes,planes, kernel_size = 3,
                               stride = 1, padding=1,bias =False)
        self.bn2 = nn.BatchNorm2d(planes) #정규화 함수이다.
        self.shortcut = nn.Sequential() #ResNet과 일반적인 Convolution 신경망의 차이는 shortcut의 유무이다.
        if stride !=1 or in_planes!=planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes,planes,kernel_size = 1, stride = stride, bias=False),
                nn.BatchNorm2d(planes))
    
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        #컨볼루션을 거친 후 숏컷으로 초기 정보를 이어붙이며 정보의 유실을 막을 수 있다.
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [6]:
class ResNet(nn.Module):
    def __init__(self,num_classes = 10):
        super(ResNet,self).__init__()
        self.in_planes=16
        
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,
                               stride = 1,padding =1,bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16,2,stride=1) #블록으로 이루어진 층이라고 생각!!
        self.layer2 = self._make_layer(32,2,stride=2)
        self.layer3 = self._make_layer(64,2,stride=2)
        self.linear = nn.Linear(64,num_classes)
    
    #코드를 읽어보면 이해할 수 있을것...!!
    def _make_layer(self,planes, num_blocks, stride):
        strides = [stride]+[1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes,stride))#채널이 블록을 거치면서 바뀜
            self.in_planes = planes #채널 맞춰주기
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out,8) #8*8차원에 8*8마스크를 씌우면 에버리지가 1개 나옴!! 채널이 64개니까 1*64개의 값들이 나옴
        out = out.view(out.size(0),-1) #펴주고
        out = self.linear(out) #마지막 레이어!!
        return out

In [7]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,momentum=0.9,weight_decay=0.0005)
#weight_decay = 가중치 감소 오버피팅을 막기 위한 방법!!
#값이 너무 크면 언더피팅이 되므로 적절한 숫자를 사용해야 함!

scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=50,gamma=0.1)
#학습률 스케줄러 함수 - 에포크가 50마다 학습률에 감마값을 곱해 학습률을 낮춘다. (정교한 최적화가 가능!!)

In [8]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data,target = data.to(DEVICE),target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()

In [9]:
def evaluate(model,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(DEVICE),target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /=len(test_loader.dataset)
    test_accuracy = 100.*correct/len(test_loader.dataset)
    return test_loss,test_accuracy

In [10]:
for epoch in range(1,EPOCHS+1):
    
    train(model, train_loader,optimizer, epoch)
    test_loss, test_accuracy = evaluate(model,test_loader)
    scheduler.step()
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss,test_accuracy))

[1] Test Loss: 1.6700, Accuracy: 43.94%
[2] Test Loss: 1.0342, Accuracy: 63.23%
[3] Test Loss: 1.1096, Accuracy: 63.02%
[4] Test Loss: 0.9667, Accuracy: 67.47%
[5] Test Loss: 0.7976, Accuracy: 73.31%
[6] Test Loss: 0.8817, Accuracy: 70.01%
[7] Test Loss: 1.0680, Accuracy: 65.25%
[8] Test Loss: 0.6918, Accuracy: 76.29%
[9] Test Loss: 0.8107, Accuracy: 72.84%
[10] Test Loss: 0.7032, Accuracy: 76.14%
[11] Test Loss: 0.6987, Accuracy: 76.11%
[12] Test Loss: 0.7445, Accuracy: 74.23%
[13] Test Loss: 0.7576, Accuracy: 75.73%
[14] Test Loss: 0.7172, Accuracy: 76.73%
[15] Test Loss: 0.6074, Accuracy: 79.15%
[16] Test Loss: 0.7032, Accuracy: 76.42%
[17] Test Loss: 1.0110, Accuracy: 69.65%
[18] Test Loss: 0.7756, Accuracy: 74.17%
[19] Test Loss: 0.7374, Accuracy: 75.20%
[20] Test Loss: 0.6574, Accuracy: 77.76%
[21] Test Loss: 0.9144, Accuracy: 71.88%
[22] Test Loss: 0.6309, Accuracy: 79.02%
[23] Test Loss: 0.8681, Accuracy: 72.85%
[24] Test Loss: 0.6096, Accuracy: 79.26%
[25] Test Loss: 0.9411, A

원래는 에포크를 늘려야 하지만 내 컴퓨터가 느려 에포크를 늘리면 학습이 내일 끝남...ㅠㅠ
---